# Quadratische Diskriminantenanalyse

In dieser Übung werden Sie selbst eine quadratische Diskriminantenanalyse (QDA) implementieren. Zur Erinnerung: Die QDA berechnet $p(x|y)=\frac{p(y|x)*p(x)}{p(y)}$. Die Likelihood $p(y|x)$ wird als normalverteilt angenommen.

## Aufgabe 1
Eine Fischerin benötigt Ihre Hilfe bei der Klassifikation von Fischen. Vor kurzem hat sie folgende Fische gefangen: 

| Länge (m)    | Art          | 
| ------------- |-------------  |
| 1.3           | Barsch       |
| 0.7           | Lachs       |
| 0.62           | Lachs      |
| 0.9           | Lachs       |
| 0.91          | Barsch       |
| 0.31          | Hering       |
| 0.26           | Hering       |

* Berechnen Sie die Priors $p(\omega)$ für jede Fischart
* Berechnen Sie die Parameter $\mu$ und $\sigma^2$ für die Lkelihoods $p(x|\omega)$. 
* Die Fischerin fängt einen neuen Fisch mit Länge $x = 0.82 m$. Berechen Sie die Posterior-Wahrscheinlichkeit $p(\omega|x)$ für jede Klasse. Wie wird der Fisch klassifiziert?

In [1]:
p_barsch = 2/7
p_lachs = 3/7
p_hering = 2/7

# Das sind die Schulvarianten von Mittelwert und Varianz. Nicht die Bias-bereinigten
mean_barsch = (1.3 + 0.91) / 2
mean_lachs = (0.7 + 0.62 + 0.9) / 3
mean_hering = (0.31 + 0.26) / 2

var_barsch = 1/2 * (1.3 - mean_barsch) ** 2 + (0.91 - mean_barsch) ** 2
var_lachs = 1/3 * (0.7 - mean_lachs) ** 2 + (0.62 - mean_lachs) ** 2 + (0.9 - mean_lachs) ** 2
var_hering = 1/2 * (0.31 - mean_hering) ** 2 + (0.26 - mean_hering) ** 2

In [2]:
import numpy as np
def gaussian(x, mean, var):
    return 1./(np.sqrt(2.*np.pi*var))*np.exp(-np.power((x - mean), 2.)/(2*var))

In [4]:
x = 0.82

t_barsch = gaussian(x, mean_barsch, var_barsch) * p_barsch
t_lachs = gaussian(x, mean_lachs, var_lachs) * p_lachs
t_hering = gaussian(x, mean_hering, var_hering) * p_hering

p_x = sum([t_barsch, t_lachs, t_hering])

w_barsch = t_barsch / p_x
w_lachs = t_lachs / p_x
w_hering = t_hering / p_x

print(t_barsch, t_lachs, t_hering, p_x, sep="\n")

print(w_barsch, w_lachs, w_hering, sum([w_barsch, w_lachs, w_hering]), sep="\n")
#Der Fisch wird also als Lachs klassifiziert

0.23416954059335204
0.7847669667187298
1.8808582763433165e-66
1.0189365073120817
0.2298175979689676
0.7701824020310325
1.8459033147266004e-66
1.0


## Aufgabe 2
Implementieren Sie eine Funktion `priors(classes)`, die für einen Vektor von Klassen-Labels den Prior $p(x)$ für jede Klasse ausgibt.
Die Eingabe soll ein Array von Klassen sein (z.b. `np.array(["stand","sit","sit","stand"])`). Die Ausgabe soll ein Data Frame mit den Spalten `class` und `prior` sein.

In [2]:
import numpy as np
import pandas as pd

def priors(classes):
    unique, counts = np.unique(classes, return_counts=True)
    counts = counts/counts.sum()
    df = pd.DataFrame(np.array([unique,counts]).transpose(),columns=["class","prior"])
    return(df)
    
pp = priors(np.array(["stand","sit","sit","sit","stand"]))
print(pp)
np.array(pp["class"])

   class prior
0    sit   0.6
1  stand   0.4


array(['sit', 'stand'], dtype=object)

## Aufgabe 3
Implementieren Sie eine Funktion `likelihood(data)`, die für ein Data Frame, bestehend aus einer Spalte $y$ und einer Spalte $x$, die Likelihood $p(y|x)$ für jede Klasse $x$ mit einer Normalverteilung approximiert, d.h. es soll für jede Klasse ein Mittelwert und eine Varianz ausgegeben werden.
Die Ausgabe soll also die Spalten `class`, `mean` und `variance` besitzen.

Plotten Sie die Likelihood für jede Klasse.

In [3]:
def likelihood(data):
    uc = np.unique(data["class"])
    def getMV(c):
        dsub = data.loc[data["class"]==c,"x"]
        return([dsub.mean(),dsub.var()])
    mvs = list(map(getMV, uc))
    r = pd.DataFrame(mvs,columns = ["mean","variance"])
    r["class"] = uc
    return r
    
data = arff.loadarff('features1.arff')
df = pd.DataFrame(data[0])

dat = df.loc[:, ["AccX_mean","class"]]
dat.columns = ["x","class"]
lik = likelihood(dat)
lik

,mean,variance,class
0,523.545587,15.843081,b'bike'
1,517.316002,18.523919,b'downstairs'
2,467.870175,56.697558,b'lie'
3,527.009294,96.483175,b'run'
4,486.721946,71.790221,b'sit'
5,528.594727,1.057665,b'stand'
6,545.413484,62.520043,b'upstairs'
7,525.170962,1.221023,b'walk'


## Aufgabe 4
Implementieren Sie eine Funktion mylda(newdat,lik,priors), die für eine neue Beobachtung `newdat` die wahrscheinlichste Klasse zurückgibt.

Testen Sie Ihre Implementierung auf dem Datensatz `features1.arff`. „Trainieren“ Sie die QDA (d.h. berechnen Sie likelihood und prior), und führen Sie dann für die gleichen Daten eine Klassifikation durch. Wie gut ist die Klassifikation?

In [12]:
from scipy.io import arff
import scipy.stats

def mylda(newdat,lik,prior):
    #Berechne für jeden Datenpunkt: Wahrscheinlichkeit pro Klasse als prior*likelihood, waehle wahrscheinlichste
    def getClass(d):
        def getProb(c,d):
            pr = prior.loc[prior["class"]==c,"prior"].values[0]
            m = lik.loc[lik["class"] == c,"mean"].values[0]
            v = lik.loc[lik["class"] == c,"variance"].values[0]
            li = scipy.stats.norm(m, np.sqrt(v)).pdf(d)
            return(li*pr)
        probs = np.array([getProb(c,d) for c in prior["class"].values])
        return prior.loc[np.argmax(probs),"class"]
    newclasses = np.array([getClass(dat) for dat in newdat])
    return newclasses


data = arff.loadarff('features1.arff')
df = pd.DataFrame(data[0])

dat = df.loc[:, ["AccX_mean","class"]]
dat.columns = ["x","class"]

lik = likelihood(dat)
prior = priors(dat["class"])

nc = mylda(dat["x"],lik,prior)
print(sum(nc == dat["class"])/len(dat))

0.6638418079096046


In [11]:
scipy.stats.norm?